#  Gathering information about publications  




 This notebook alowes us to visit site [arxiv.org](https://arxiv.org) and scrape information about different publications 
in physics. In particular we are interested in the following topics: 
* Black Holes and General Relativity(GR)
* Tensor Networks
* Holography (aka AdS/CFT)
* Quantum Computers ans Quantum Infomation


  
  
  


In [41]:
from docx import Document
import re
import requests
from bs4 import BeautifulSoup
import numpy as np

In [42]:
                            ##  We adjust list of key words for articles ##
    
def read_docx(file_path):
    doc = Document(file_path)
    text = []
    for paragraph in doc.paragraphs:
        # Only append non-empty strings
        if paragraph.text.strip():
            text.append(paragraph.text.strip())
    return text

key_words_file_path = 'articles_to_analyse/words_for_search.docx'
key_words_list = read_docx(key_words_file_path)

# if  we want to add some words but we don't want to modify docx file right now:
words_to_add = ['traversable wormhole',
                'RT formula', 'random quantum circuits',
                'quantum circuit', 
                'black holes',
                'matrix product state',
                'large central charge',
                'quantum chaos'
                ]
key_words_list.extend(words_to_add)
for word in sorted(key_words_list):
    print(word)

AdS/CFT
BTZ black hole
BTZ black holes
MERA
RT formula
RT formula
Renyi entropy
bit thread
black holes
bulk geometry
cMERA
continuous tensor network
einstein equations
emergent space-time geometry
entanglement of assistance
entanglement of purification
entanglement renormalization
geodesics
geometry of entangled states
hawking radiation
holographic code
holographic duality
holographic entanglement
holographic shadow
holography
hyperbolic lattices
large central charge
matrix product state
quantum bit threads
quantum chaos
quantum circuit
quantum computer
quantum error-correcting code
quantum information theory
quantum machine learning
random quantum circuits
stabilizer state
tensor network
tensor networks
tensor networks in machine learning
traversable wormhole
traversable wormholes


In [43]:
def total_number_of_articles(to_date, from_date):
    

    url = 'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date='+ from_date +'&date-to_date='+ to_date +'&date-date_type=submitted_date&abstracts=show&size=200&order=-announced_date_first&start=0' 

    # Send an HTTP GET request to the webpage
    response = requests.get(url)

        # Check if the request was successful (status code 200)
    if response.status_code == 200:
            # Get the HTML content of the webpage
        arxiv_page_content = response.text
    else:
        print(f"Failed to fetch webpage. Status code: {response.status_code}")


    # Regular expression pattern to extract the desired number
    pattern = r'Showing \d+&ndash;\d+ of (\d{1,3}(?:,\d{3})*) results'

    # Search for the pattern in the content
    match = re.search(pattern, arxiv_page_content)

    # Extract the number if match found
    if match:
        result = int(match.group(1).replace(',', ''))
        return result 
    else:
        return 0 

 



to_date = '2024-06-22'    # 'YYYY-MM-DD'
from_date = '2024-01-01'  # 'YYYY-MM-DD'
    
print(f"The overall number of articles released from {from_date} till {to_date} is: {total_number_of_articles(to_date, from_date)}")    
    
    
    

The overall number of articles released from 2024-01-01 till 2024-06-22 is: 51739


In [44]:

def fetching_webpage(from_date, to_date, starting_num):
    
    
        ## This function returns string that contains Page source code for  advanced search for physics articles for 200 articles starting from starting_num
        url = 'https://arxiv.org/search/advanced?advanced=&terms-0-operator=AND&terms-0-term=&terms-0-field=title&classification-physics=y&classification-physics_archives=all&classification-include_cross_list=include&date-year=&date-filter_by=date_range&date-from_date='+ from_date +'&date-to_date='+ to_date +'&date-date_type=submitted_date&abstracts=show&size=200&order=-announced_date_first&start='+ str(starting_num) 

        # Send an HTTP GET request to the webpage
        response = requests.get(url)

        # Check if the request was successful (status code 200)
        if response.status_code == 200:
            # Get the HTML content of the webpage
            arxiv_page_content = response.text
        else:
            print(f"Failed to fetch webpage. Status code: {response.status_code}")    
            
        return arxiv_page_content     

        
    

In [45]:
def find_last_date_on_a_webpage(arxiv_page_content, table_arxiv):
    
    
        yy = table_arxiv[-1][0][22:24]
        mm = table_arxiv[-1][0][24:26]

        # Define the pattern
        pattern = r'<span class="has-text-black-bis has-text-weight-semibold">Submitted</span>.*?<span class="has-text-black-bis has-text-weight-semibold">originally announced</span>'

        # Find all matches of the pattern
        matches = re.finditer(pattern, arxiv_page_content, re.DOTALL)

        # Get the last match
        last_match = None
        for match in matches:
            last_match = match
        answer = last_match.group()


        # Define the pattern to extract the date
        pattern = r'<span class="has-text-black-bis has-text-weight-semibold">Submitted</span>\s*(.*?)\s*;'

        # Search for the pattern in the text
        match = re.search(pattern, answer)

        # If a match is found, extract the date
        if match:
           date_text = match.group(1)
           # Extract the day part from the date
           date_match = re.search(r'\d+', date_text)
           if date_match:
              dd = date_match.group()    
    
        new_to_date = '20'+ str(yy) + '-' + str(mm) + '-' + str(dd)
        
        return new_to_date
   
    

In [46]:
"""  
  returns: arxiv_table =  [['link_to_article',
                            'name_of_the_article', 
                            'author1, author2,..', 
                            'Abstract_full_version',
                            {key_words_1: n_1, key_words_2: n_2,...}],
                            ...]


  example of an article from arxiv_table: 
            arxiv_table[i] = ['https://arxiv.org/pdf/2403.07275', 
             
                              'Coarse-graining black holes out of equilibrium with boundary observables on time slice',
                              
                              'Daichi Takeda',
                              
                              'In black hole thermodynamics, defining coarse-grained entropy have been explored.
                               Guided by the AdS/CFT, we introduce a new definition of coarse-grained entropy.The entropy is shown 
                               to satisfy a generalized first law and, through holography, the second law as well. 
                               By applying this thermodynamics to several Vaidya models in AdS and flat spacetime, we discover a 
                               connection between the second law and the null energy condition.△ Less', 
                               
                               {'AdS/CFT': 1, 'holography': 1}]
     
 

"""

def create_table_arxiv(arxiv_page_content, key_words_list):
    table_arxiv = []
    soup = BeautifulSoup(arxiv_page_content, 'html.parser')

    # Define the regex pattern to match the IDs
    id_pattern = re.compile(r'(\d{4}\.\d{5}(v\d+)?)')
    
    # Find all occurrences of the title, author, and abstract patterns
    titles = re.findall(r'<p class="title is-5 mathjax">\s*(.*?)\s*</p>', arxiv_page_content)
    author_blocks = re.findall(r'<p class="authors">(.*?)</p>', arxiv_page_content, re.DOTALL)
    abstract_full_elements = soup.find_all('span', class_='abstract-full')

    # Initialize authors list and abstracts list
    authors_list = []
    abstracts = []

    # Extract authors using BeautifulSoup instead of regex
    for block in soup.find_all('p', class_='authors'):
        authors = [a.text for a in block.find_all('a')]
        formatted_authors = ", ".join(authors)
        authors_list.append(formatted_authors)
    
    # Extract abstracts
    for element in abstract_full_elements:
        abstracts.append(element.get_text(strip=True))

    # Extract IDs
    extracted_ids = []
    for span in soup.find_all('span', {'class': 'abstract-short'}):
        if span.has_attr('id'):
            match = id_pattern.search(span['id'])
            if match:
                extracted_ids.append(match.group(1))
    
    # Iterate over the extracted elements
    for i in range(max(len(extracted_ids), len(titles), len(authors_list), len(abstracts))):
        sublist = []
        sublist.append(extracted_ids[i] if i < len(extracted_ids) else np.nan)
        sublist.append(titles[i] if i < len(titles) else np.nan)
        sublist.append(authors_list[i] if i < len(authors_list) else np.nan)
        sublist.append(abstracts[i] if i < len(abstracts) else np.nan)

        # If the abstract is a string, search for keywords
        if isinstance(sublist[3], str):
            word_counts = {}
            for word in key_words_list:
                pattern = re.compile(r'\b' + re.escape(word) + r'\b', re.IGNORECASE)
                matches = pattern.findall(sublist[3])
                count = len(matches)
                if count != 0:
                    word_counts[word] = count

            sorted_word_counts = dict(sorted(word_counts.items(), key=lambda item: item[0]))
            if len(sorted_word_counts) != 0:
                sublist.append(sorted_word_counts)
                table_arxiv.append(sublist)  # Append only if sorted_word_counts is non-zero
                
                
                
    for article in table_arxiv:     
        article[0] = 'https://arxiv.org/pdf/' + str(article[0]) # Convert 2403.07972v1 to https://arxiv.org/pdf/2403.07972v1
        article[3] = article[3][:-6]   # Delete "△ Less" from the end of every Abstract# 
        
    return table_arxiv




In [47]:
### This fucntion puts together the everything that needs to be done in order to get info about the articles
### Note that advanced search at arxiv.org shows no more than 200 publications per page
### Total number of articles that can be scrolled this way is no more than 10000
### If we want to go through more than 10000 publications, which is usually the case we need to take these issues into account



def THE_FUNCTION(from_date, to_date, resulting_table=None):
    if resulting_table is None:
        resulting_table = []
    
    total_num_of_articles = total_number_of_articles(to_date, from_date)
    
    if total_num_of_articles <= 10000:
        for starting_num in range(0, total_num_of_articles, 200):
            print(starting_num)
            arxiv_page_content = fetching_webpage(from_date, to_date, starting_num)
            resulting_table.extend(create_table_arxiv(arxiv_page_content, key_words_list))
            print(f"number of interesting articles: {len(resulting_table)}")
    else:
        for starting_num in range(0, 10000, 200):
            print(starting_num)
            arxiv_page_content = fetching_webpage(from_date, to_date, starting_num)
            resulting_table.extend(create_table_arxiv(arxiv_page_content, key_words_list))
            print(f"number of interesting articles: {len(resulting_table)}")

            if starting_num == 9800:
                new_to_date = find_last_date_on_a_webpage(arxiv_page_content, resulting_table)
                return THE_FUNCTION(from_date, new_to_date, resulting_table)  # Pass the current table for updating
    
    return resulting_table
    







In [55]:
to_date = '2024-06-22'    # 'YYYY-MM-DD'
from_date = '2024-01-01'  # 'YYYY-MM-DD'

arxiv_table_2024 = THE_FUNCTION(from_date, to_date)

### To keep track of things we print overall number of "interesting" articles obtained based on the list of key words


## 0
## number of interesting articles: 4  
## 200 <-- (number of  articles per page)
## number of interesting articles: 5
## 400 <--
## number of interesting articles: 23
## 600 <--



0
number of interesting articles: 4
200
number of interesting articles: 5
400
number of interesting articles: 23
600
number of interesting articles: 33
800
number of interesting articles: 47
1000
number of interesting articles: 61
1200
number of interesting articles: 80
1400
number of interesting articles: 93
1600
number of interesting articles: 104
1800
number of interesting articles: 119
2000
number of interesting articles: 133
2200
number of interesting articles: 146
2400
number of interesting articles: 157
2600
number of interesting articles: 174
2800
number of interesting articles: 186
3000
number of interesting articles: 199
3200
number of interesting articles: 209
3400
number of interesting articles: 220
3600
number of interesting articles: 234
3800
number of interesting articles: 249
4000
number of interesting articles: 268
4200
number of interesting articles: 276
4400
number of interesting articles: 289
4600
number of interesting articles: 305
4800
number of interesting articl

number of interesting articles: 2304
9800
number of interesting articles: 2322
0
number of interesting articles: 2330
200
number of interesting articles: 2335
400
number of interesting articles: 2344
600
number of interesting articles: 2349
800
number of interesting articles: 2352
1000
number of interesting articles: 2365
1200
number of interesting articles: 2376
1400
number of interesting articles: 2385
1600
number of interesting articles: 2397
1800
number of interesting articles: 2414
2000
number of interesting articles: 2428
2200
number of interesting articles: 2448
2400
number of interesting articles: 2463


###  Next couple of cells deal with different kinds of problems in cereral items

  
  

In [56]:
##For this article the was a glitch with article[0]

for article in arxiv_table_2024:    
    if article[1]== 'Geometrical methods in mathematical physics':
       article[0] = "https://arxiv.org/pdf/1311.0733.pdf"

In [57]:
### Create additional columns 'mm.yyyy', 'yyyy', 'mm'  to newly created table


for i in range(len(arxiv_table_2024)):
    article_id = str(arxiv_table_2024[i][0])  # Convert to string to avoid TypeError
    date = article_id[24:26] + '.20' + article_id[22:24]
    year = '20' + article_id[-10:-8]
    month = article_id[-8:-6]
    arxiv_table_2024[i].append(date)
    arxiv_table_2024[i].append(date[-4:])
    arxiv_table_2024[i].append(date[:2])
    


In [58]:
# Define your keyword categories
categories = {
    'Tensor_Networks': [
        'tensor network', 'tensor networks', 'MERA', 'random tensor network', 'RTN', 
        'cMERA', 'hyperbolic lattices', 'continuous tensor network',
        'tensor networks in machine learning', 'stabilizer state'
    ],
    'Holography': [
        "AdS/CFT", "BTZ black hole", "BTZ black holes", "RT formula", "bulk geometry",
        "holographic code", "holographic duality", "quantum error-correcting code",
        "holographic entanglement", "holographic shadow", "holography", "Renyi entropy",
        "bit thread", "quantum bit threads", "emergent space-time geometry",
        "geometry of entangled states"
    ],
    'Black_Holes_and_GR': [
        "hawking radiation", "geodesics", "einstein equations", "traversable wormhole",
        "traversable wormholes", "black holes"
    ],
    'Quantum_Computers_and_Quantum_Information': [
        "entanglement of assistance", "entanglement of purification", "entanglement renormalization",
        "quantum circuit", "quantum computer", "quantum error-correcting code", "quantum information theory",
        "quantum machine learning", "tensor networks in machine learning", "random quantum circuits"
    ]
}

def categorize_articles(articles, categories):
    for article in articles:
        # Extract the keywords dictionary from the article
        keywords_dict = article[4]
        # Initialize a list to store categories for this article
        article_categories = []
        
        # Iterate over each category and its associated keywords
        for category_name, keywords_list in categories.items():
            # Convert the category name into a properly spaced string
            # only replacing the first occurrence of '_and_' with ' and '
            formatted_category_name = category_name.replace('_', ' ').replace(' and ', ' and ', 1)
            
            # Check if any keyword from the current category is in the article's keyword dictionary
            if any(keyword for keyword in keywords_list if keyword in keywords_dict):
                # If any keyword matches, append the properly formatted category name to the article's categories
                article_categories.append(formatted_category_name)
        
        # Replace or append the new categories list to the existing article
        if len(article) > 8:  # If categories were already assigned, replace them
            article[8] = article_categories
        else:  # Otherwise, append the new categories list
            article.append(article_categories)

# Apply the updated function to your articles
categorize_articles(arxiv_table_2024, categories)

In [59]:
unique_articles = []
unique_urls = set()  # A set to track URLs of articles we've already added

for article in arxiv_table_2024:
    if article[0] not in unique_urls:  # Check if the article's URL is new
        unique_articles.append(article)  # Add the article to the list of unique articles
        unique_urls.add(article[0])  # Mark this URL as seen

arxiv_table_2024 = unique_articles  # Replace the original list with the list of unique articles

In [60]:
##For this article the was a glitch with article[3]

for article in arxiv_table_2024:    
    if article[1]== 'Detecting Measurement-Induced Entanglement Transitions With Unitary Mirror Circuits':
       print(article) 
       article[3] = "Monitored random circuits, consisting of alternating layers of entangling two-qubit gates and projective single-qubit measurements applied to some fraction $p$ of the qubits, have been a topic of recent interest. In particular, the resulting steady state exhibits a phase transition from highly correlated states with &#34;volume-law&#34; entanglement at $p&lt;p_{c}$ to localized states with &#34;area-law&#34; entanglement at $p&gt;p_{c}$. It is hard to access this transition experimentally, as it cannot be seen at the ensemble level. Naively, to observe it one must repeat the experiment until the set of measurement results repeats itself, with likelihood that is exponentially small in the number of measurements. To overcome this issue, we present a hybrid quantum-classical algorithm which creates a matrix product state (MPS) based &#34;unitary mirror&#34; of the projected circuit. Polynomial-sized tensor networks can represent quantum states with area-law entanglement, and so the unitary mirror can well-approximate the experimental state above $p_{c}$ but fails exponentially below it. The breaking of this mirror can thus pinpoint the critical point. We outline the algorithm and how such results would be obtained. We present a bound on the maximum entanglement entropy of any given state that is well-represented by an MPS, and from the bound suggest how the volume-law phase can be bounded. We consider whether the entanglement could similarly be bounded from below where the MPS fails. Finally, we present numerical results for small qubit numbers and for monitored circuits with random Clifford gates."
       print('\n',article) 

In [62]:
print(len(arxiv_table_2024))
%store arxiv_table_2024

1970
Stored 'arxiv_table_2024' (list)


In [63]:
import pandas as pd

# Convert the list of lists to a pandas DataFrame
df = pd.DataFrame(arxiv_table_2024, columns=['Link', 'Title', 'Authors', 'Abstract', 'Keywords', 'Date', 'Year', 'Month', 'Topic'])

# Set the index to the titles
df.set_index('Title', inplace=True)

# Apply some formatting and styling
styled_df = df.style \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }, {
        'selector': 'tr:nth-of-type(odd)',
        'props': [('background-color', 'lightgray')]
    }])

%store df







Stored 'df' (DataFrame)


In [65]:
## If we show this way thw whole table (987 items) the notebook slows down

df_slice = pd.DataFrame(arxiv_table_2024[:2], columns=['Link', 'Title', 'Authors', 'Abstract', 'Keywords', 'Date', 'Year', 'Month', 'Topic'])

# Set the index to the titles
df_slice.set_index('Title', inplace=True)

# Apply some formatting and styling
styled_df_slice = df_slice.style \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }, {
        'selector': 'tr:nth-of-type(odd)',
        'props': [('background-color', 'lightgray')]
    }])

styled_df_slice 

,Link,Authors,Abstract,Keywords,Date,Year,Month,Topic
Title,,,,,,,,
Parameterized quasinormal frequencies and Hawking radiation for axial gravitational perturbations of a holonomy-corrected black hole,https://arxiv.org/pdf/2406.15711v1,"Sen Yang, Wen-Di Guo, Qin Tan, Li Zhao, Yu-Xiao Liu","As the fingerprints of black holes, quasinormal modes are closely associated with many properties of black holes. Especially, the ringdown phase of gravitational waveforms from the merger of compact binary components can be described by quasinormal modes. Serving as a model-independent approach, the framework of parameterized quasinormal frequencies offers a universal method for investigating quasinormal modes of diverse black holes. In this work, we first obtain the Schrödinger-like master equation of the axial gravitational perturbation of a holonomy-corrected black hole. We calculate the corresponding quasinormal frequencies using the Wentzel-Kramers-Brillouin approximation and asymptotic iteration methods. We investigate the numerical evolution of an initial wave packet on the background spacetime. Then, we deduce the parameterized expression of the quasinormal frequencies and find that $r_0 \leq 10^{-2}$ is a necessary condition for the parameterized approximation to be valid. We also study the impact of the quantum parameter $r_0$ on the greybody factor and Hawking radiation. With more ringdown signals of gravitational waves detected in the future, our research will contribute to the study of the quantum properties of black holes.","{'black holes': 4, 'hawking radiation': 1}",06.2024,2024,06,['Black Holes and GR']
Observation of a non-Hermitian supersonic mode,https://arxiv.org/pdf/2406.15557v1,"Yuxuan Zhang, Juan Carrasquilla, Yong Baek Kim","Quantum computers have long been anticipated to excel in simulating quantum many-body physics. While most previous work has focused on Hermitian physics, we demonstrate the power of variational quantum circuits for resource-efficient simulations of dynamical and equilibrium physics in non-Hermitian systems, revealing new phenomena beyond standard Hermitian quantum machines. Using a variational quantum compilation scheme for fermionic systems, we reduce gate count, save qubits, and eliminate the need for postselection, a major challenge in simulating non-Hermitian dynamics via standard Trotterization. Experimentally, we observed a supersonic mode in the connected density-density correlation function on an $ n = 18 $ fermionic chain after a non-Hermitian, locally interacting quench, which would otherwise be forbidden by the Lieb-Robinson bound in a Hermitian system. Additionally, we investigate sequential quantum circuits generated by tensor networks for ground state preparation, here defined as the eigenstate with the lowest real part eigenvalue, using a variance minimization scheme. Through a trapped-ion implementation on the Quantinuum H1 quantum processor, we accurately capture correlation functions and energies across an exceptional point on a dissipative spin chain up to length $ n = 20 $ using only 3 qubits. Motivated by these advancements, we provide an analytical example demonstrating that simulating single-qubit non-Hermitian dynamics for $Θ(\log(n))$ time from certain initial states is exponentially hard on a quantum computer, offering insights into the opportunities and limitations of using quantum computation for simulating non-Hermitian physics.","{'quantum computer': 1, 'tensor networks': 1}",06.2024,2024,06,"['Tensor Networks', 'Quantum Computers and Quantum Information']"


In [66]:
# Assuming df is your DataFrame
df.to_csv('table_arxiv_24-01-01__2024-06-22.csv', index=True)

In [67]:
print(len(df))

1970


 # This is essentially the end of the project.  
 ## The rest of the notebook splits this overall table into several smaller ones based on Topics.  










In [68]:
TN_table_2024 = []  # List to hold articles related to Tensor Networks

for article in arxiv_table_2024:
    # Check if 'Tensor Networks' is in the list of topics (which is the last element of the article)
    if 'Tensor Networks' in article[-1]:
        TN_table_2024.append(article)  # Add the whole article to the TN_table_2024 list

In [69]:
print(len(TN_table_2024))
%store TN_table_2024

143
Stored 'TN_table_2024' (list)


In [70]:
Holography_table_2024 = []  # List to hold articles related to Holography

for article in arxiv_table_2024:
    # Check if 'Holography' is in the list of topics (which is the last element of the article)
    if 'Holography' in article[-1]:
        Holography_table_2024.append(article)  # Add the whole article to the Holography_table_2024 list
        

In [71]:
print(len(Holography_table_2024))

185


In [72]:
# Convert the list of lists to a pandas DataFrame
df_TN = pd.DataFrame(TN_table_2024[:2], columns=['Link', 'Title', 'Authors', 'Abstract', 'Keywords', 'Date', 'Year', 'Month', 'Topic'])

# Set the index to the titles
df_TN.set_index('Title', inplace=True)

# Apply some formatting and styling
styled_df_TN = df_TN.style \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }, {
        'selector': 'tr:nth-of-type(odd)',
        'props': [('background-color', 'lightgray')]
    }])

#%store df

# Display the styled DataFrame
styled_df_TN

,Link,Authors,Abstract,Keywords,Date,Year,Month,Topic
Title,,,,,,,,
Observation of a non-Hermitian supersonic mode,https://arxiv.org/pdf/2406.15557v1,"Yuxuan Zhang, Juan Carrasquilla, Yong Baek Kim","Quantum computers have long been anticipated to excel in simulating quantum many-body physics. While most previous work has focused on Hermitian physics, we demonstrate the power of variational quantum circuits for resource-efficient simulations of dynamical and equilibrium physics in non-Hermitian systems, revealing new phenomena beyond standard Hermitian quantum machines. Using a variational quantum compilation scheme for fermionic systems, we reduce gate count, save qubits, and eliminate the need for postselection, a major challenge in simulating non-Hermitian dynamics via standard Trotterization. Experimentally, we observed a supersonic mode in the connected density-density correlation function on an $ n = 18 $ fermionic chain after a non-Hermitian, locally interacting quench, which would otherwise be forbidden by the Lieb-Robinson bound in a Hermitian system. Additionally, we investigate sequential quantum circuits generated by tensor networks for ground state preparation, here defined as the eigenstate with the lowest real part eigenvalue, using a variance minimization scheme. Through a trapped-ion implementation on the Quantinuum H1 quantum processor, we accurately capture correlation functions and energies across an exceptional point on a dissipative spin chain up to length $ n = 20 $ using only 3 qubits. Motivated by these advancements, we provide an analytical example demonstrating that simulating single-qubit non-Hermitian dynamics for $Θ(\log(n))$ time from certain initial states is exponentially hard on a quantum computer, offering insights into the opportunities and limitations of using quantum computation for simulating non-Hermitian physics.","{'quantum computer': 1, 'tensor networks': 1}",06.2024,2024,06,"['Tensor Networks', 'Quantum Computers and Quantum Information']"
Tensor Decompositions and Adiabatic Quantum Computing for Discovering Practical Matrix Multiplication Algorithms,https://arxiv.org/pdf/2406.13412v1,Valter Uotila,"Quantum computing and modern tensor-based computing have a strong connection, which is especially demonstrated by simulating quantum computations with tensor networks. The other direction is less studied: quantum computing is not often applied to tensor-based problems. Considering tensor decompositions, we focus on discovering practical matrix multiplication algorithms and develop two algorithms to compute decompositions on quantum computers. The algorithms are expressed as higher-order unconstrained binary optimization (HUBO) problems, which are translated into quadratic unconstrained binary optimization (QUBO) problems. Our first algorithm is decompositional to keep the optimization problem feasible for the current quantum devices. Starting from a suitable initial point, the algorithm discovers tensor decomposition corresponding to the famous Strassen matrix multiplication algorithm, utilizing the current quantum annealers. Since the decompositional algorithm does not guarantee minimal length for found tensor decompositions, we develop a holistic algorithm that can find fixed-length decompositions. Theoretically, by fixing a shorter length than the length for the best-known decomposition, we can ensure that the solution to the holistic optimization problem would yield faster matrix multiplication algorithms.",{'tensor networks': 1},06.2024,2024,06,['Tensor Networks']


In [73]:
# Convert the list of lists to a pandas DataFrame
df_Holography = pd.DataFrame(Holography_table_2024[:1], columns=['Link', 'Title', 'Authors', 'Abstract', 'Keywords', 'Date', 'Year', 'Month', 'Topic'])

# Set the index to the titles
df_Holography.set_index('Title', inplace=True)

# Apply some formatting and styling
styled_df_Holography = df_Holography.style \
    .set_properties(**{'text-align': 'left'}) \
    .set_table_styles([{
        'selector': 'th',
        'props': [('text-align', 'left')]
    }, {
        'selector': 'tr:nth-of-type(odd)',
        'props': [('background-color', 'lightgray')]
    }])

#%store df

# Display the styled DataFrame
styled_df_Holography

,Link,Authors,Abstract,Keywords,Date,Year,Month,Topic
Title,,,,,,,,
Phase space framework enables a variable-scale diffraction model for coherent imaging and display,https://arxiv.org/pdf/2406.15456v1,"Zhi Li, Xuhao Luo, Jing Wang, Xin Yuan, Dongdong Teng, Qiang Song, Huigao Duan","The fast algorithms in Fourier optics have invigorated multifunctional device design and advanced imaging technologies. However, the necessity for fast computations has led to limitations in the widely used conventional Fourier methods, manifesting as fixed size image plane at a certain diffraction distance. These limitations pose challenges in intricate scaling transformations, 3D reconstructions and full-color displays. Currently, there is a lack of effective solutions, often resorting to pre-processing that compromise fidelity. In this paper, leveraging a higher-dimensional phase space method, we present a universal framework allowing for customized diffraction calculation methods. Within this framework, we establish a variable-scale diffraction computation model which allows the adjustment of the size of the image plane and can be operated by fast algorithms. We validate the model's robust variable-scale capabilities and its aberration automatic correction capability for full-color holography, achieving high fidelity. The large-magnification tomography experiment demonstrates that this model provides a superior solution for holographic 3D reconstruction. In addition, this model is applied to achieve full-color metasurface holography with near-zero crosstalk, showcasing its versatile applicability at nanoscale. Our model presents significant prospects for applications in the optics community, such as beam shaping, computer-generated holograms (CGHs), augmented reality (AR), metasurface optical elements (MOEs) and advanced holographic head-up display (HUD) systems.",{'holography': 2},06.2024,2024,06,['Holography']
